In [25]:
from PIL import Image

In [26]:
import tensorflow as tf
from sklearn.utils import shuffle

In [27]:
BATCH_SIZE = 32

NUM_EPOCH = 50

INIT_LR = 5e-7

In [28]:
img = Image.open('static-images/1.png')

In [29]:
img = Image.open('training_data/cs10000053986926948907.png')

In [30]:
input_layer = tf.keras.layers.Input(shape=(100,100, 4))
rescale = tf.keras.layers.Rescaling(1./255, input_shape=(100, 100, 4))(input_layer)
flat_layer = tf.keras.layers.Flatten()(rescale)
hidden_layer_1 = tf.keras.layers.Dense(4000, activation='sigmoid')(flat_layer)
hidden_layer_2 = tf.keras.layers.Dense(4000, activation='sigmoid')(hidden_layer_1)
hidden_layer_3 = tf.keras.layers.Dense(400, activation='sigmoid')(hidden_layer_2)
out_later = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer_3)

critic_model = tf.keras.Model(input_layer, out_later)
#model.build((input_size,1,1))
critic_model.compile(
    #loss='sparse_categorical_crossentropy' ,
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=INIT_LR/3, beta_1=0.5, decay=INIT_LR / (NUM_EPOCH*3)),
    metrics=["accuracy"],
)

critic_model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 100, 4)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 100, 100, 4)       0         
                                                                 
 flatten_1 (Flatten)         (None, 40000)             0         
                                                                 
 dense_7 (Dense)             (None, 4000)              160004000 
                                                                 
 dense_8 (Dense)             (None, 4000)              16004000  
                                                                 
 dense_9 (Dense)             (None, 400)               1600400   
                                                                 
 dense_10 (Dense)            (None, 1)                 401 

/home/jliddy/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [31]:
type(critic_model)

keras.engine.functional.Functional

In [32]:
critic_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 100, 4)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 100, 100, 4)       0         
                                                                 
 flatten_1 (Flatten)         (None, 40000)             0         
                                                                 
 dense_7 (Dense)             (None, 4000)              160004000 
                                                                 
 dense_8 (Dense)             (None, 4000)              16004000  
                                                                 
 dense_9 (Dense)             (None, 400)               1600400   
                                                                 
 dense_10 (Dense)            (None, 1)                 401 

In [33]:
import numpy as np

In [34]:
import pathlib

In [35]:
bad_imgs = []
for img_path in  pathlib.Path('static-images').glob('*.png'):
    bad_imgs.append(np.array(Image.open(img_path)))

In [36]:
good_imgs = []
for img_path in  pathlib.Path('training_data').glob('*.png'):
    good_imgs.append(np.array(Image.open(img_path)))

In [37]:
good_imgs[3][25][25]

array([70, 46, 31,  0], dtype=uint8)

In [38]:
all_images = bad_imgs + good_imgs
all_labels = [0] * len(bad_imgs) + [1] * len(good_imgs)

In [39]:
from sklearn import model_selection

In [40]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(all_images, all_labels, test_size=.2)

In [41]:
#critic_model.fit(np.array(x_train), np.array(y_train),  validation_data=(np.array(x_test), np.array(y_test)))

In [42]:
type(critic_model)

keras.engine.functional.Functional

In [43]:
gen_input_layer = tf.keras.layers.Input(shape=(100,))
gen_hidden_layer_1 = tf.keras.layers.Dense(4000, activation='sigmoid')(gen_input_layer)
gen_hidden_layer_2 = tf.keras.layers.Dense(8000, activation='sigmoid')(gen_hidden_layer_1)
gen_hidden_layer_3 = tf.keras.layers.Dense(100*100*4, activation='sigmoid')(gen_hidden_layer_2)
gen_reshape_layer = tf.keras.layers.Reshape((100,100,4))(gen_hidden_layer_3)
gen_rescale = tf.keras.layers.Rescaling(255., input_shape=(100, 100, 4))(gen_reshape_layer)

gen_model = tf.keras.Model(gen_input_layer, gen_rescale)
#model.build((input_size,1,1))
gen_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_11 (Dense)            (None, 4000)              404000    
                                                                 
 dense_12 (Dense)            (None, 8000)              32008000  
                                                                 
 dense_13 (Dense)            (None, 40000)             320040000 
                                                                 
 reshape_1 (Reshape)         (None, 100, 100, 4)       0         
                                                                 
 rescaling_3 (Rescaling)     (None, 100, 100, 4)       0         
                                                                 
Total params: 352,452,000
Trainable params: 352,452,000
Non

In [44]:
# Turn trainable off for critic model
# Voodoo magic shit makes the critic model trainable when using itself, but not when using 
# the critic model.
critic_model.trainable = False
gan_input = tf.keras.layers.Input(shape=(100,))
gan_model_output = critic_model(gen_model(gan_input))
gan_model = tf.keras.Model(gan_input, gan_model_output)
gan_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCH))
#big_model = tf.keras.Model

In [45]:
from imutils import build_montages
import cv2
import os

In [46]:
def make_montage(image_array, length, width):
    long_images = []
    for wi in range(width):
        long_images.append(cv2.vconcat(image_array[length * wi:(length*(wi + 1))]))
    
    return Image.fromarray(cv2.hconcat(long_images))


In [47]:


# Used to generate "consistant" images
benchmarkNoise = np.random.uniform(0, 1, size=(256, 100))

for epoch in range(NUM_EPOCH):

    # get num batches
    batches_per_epoch = int(len(good_imgs)/BATCH_SIZE)

    # train one batch at a time 
    for batch_i in range(batches_per_epoch):
        # p is for saving a picture
        p = None
		
        # Get images
        image_batch = good_imgs[batch_i * BATCH_SIZE:(batch_i + 1) * BATCH_SIZE]

        # Get noise, use it to generate fake images:
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
        generated_images = gen_model.predict(noise)

        # feed take and real images to the "critic"
        critic_images = np.concatenate((image_batch, generated_images))
        critic_is_real = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
        critic_is_real = np.reshape(critic_is_real, (-1,))
        (critic_images, desc_is_real) = shuffle(critic_images, critic_is_real)

        critic_loss = critic_model.train_on_batch(critic_images, critic_is_real)


        # Finally, teach the gen model how to fool the critic.
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        fakeLabels = [1] * BATCH_SIZE
        fakeLabels = np.reshape(fakeLabels, (-1,))
        ganLoss = gan_model.train_on_batch(noise, fakeLabels)
        
		
        # Visualize the epoch
        # check to see if this is the end of an epoch, and if so,
        # initialize the output path
        if batch_i == batches_per_epoch - 1:
            p = ["gen_progress", "epoch_{}_output.png".format(
            str(epoch + 1).zfill(4))]
            # otherwise, check to see if we should visualize the current
            # batch for the epoch
        else:
        # create more visualizations early in the training
        # process
            if epoch < 10 and batch_i % 25 == 0:
                p = ["gen_progress", "epoch_{}_step_{}.png".format(
                str(epoch + 1).zfill(4), str(batch_i).zfill(5))]
                # visualizations later in the training process are less
                # interesting
            elif epoch >= 10 and batch_i % 100 == 0:
                p = ["gen_progress", "epoch_{}_step_{}.png".format(
                str(epoch + 1).zfill(4), str(batch_i).zfill(5))]

        # check to see if we should visualize the output of the
        # generator model on our benchmark data
        if p is not None:
        # show loss information
            #print("[INFO] Step {}_{}: discriminator_loss={:.6f}, "
            #"adversarial_loss={:.6f}".format(epoch + batch_i, batch_i,
            #critic_loss, ganLoss))
            # make predictions on the benchmark noise, scale it back
            # to the range [0, 255], and generate the montage
            #"adversarial_loss={:.6f}".format(epoch + batch_i, batch_i,
            #critic_loss, ganLoss))
            # make predictions on the benchmark noise, scale it back
            # to the range [0, 255], and generate the montage
            images = gen_model.predict(benchmarkNoise)
            images = images.astype('uint8')
            # write the visualization to disk
            vis = make_montage(images, 16, 16)
            p = os.path.sep.join(p)
            vis.save(p)

1/1 [==============================] - 0s 163ms/step


KeyboardInterrupt: 

In [81]:
# Step 0, initialize things to watch model step-by-step
i = 0

In [82]:

# Step 1. Generate images fake with gen model.

noise = np.random.uniform(-1, 1, size=(128, 100))
generated_images = gen_model.predict(noise)

4/4 [==============================] - 0s 7ms/step
